# Import

In [ ]:
from matplotlib.colors import SymLogNorm
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates.sky_coordinate import SkyCoord
import astropy.units as u
from astropy.cosmology import Planck15 as cosmo
from astropy.cosmology import FlatLambdaCDM
from astropy.wcs import WCS
from minot import model
import scipy

# Modify plotting parameters
dict_base = {'font.size':        16,
             'legend.fontsize':  16,
             'xtick.labelsize':  16,
             'ytick.labelsize':  16,
             'axes.labelsize':   16,
             'axes.titlesize':   16,
             'figure.titlesize': 16,    
             'figure.figsize':[8.0, 6.0],
             'figure.subplot.right':0.97,
             'figure.subplot.left':0.15,
             'font.family':'serif',
             'figure.facecolor': 'white',
             'legend.frameon': True}
plt.rcParams.update(dict_base)

# Define a cluster object

In [ ]:
cosmo_coma = FlatLambdaCDM(H0=67.8, Om0=0.308, Tcmb0=2.7255, Ob0=0.0484, Neff=3.13, m_nu=[0, 0, 0.06] * u.eV)
clust = model.Cluster(name='Coma', 
                      redshift=0.0231, M500=6.13e14*u.Msun, 
                      cosmology=cosmo_coma  , silent=False, 
                      output_dir='/home/roman/CTA/cluster/coma_main')

clust.coord = SkyCoord("194.9119d  27.9537d", frame="icrs", unit="deg")

#----- Define the relevant ICM model (available models are defined in model_modpar.py)
clust.density_gas_model = {'name':'beta', 'n_0':3.36e-3*u.cm**-3, 'beta':0.75, 'r_c':310*u.kpc}
clust.pressure_gas_model = {'name':'GNFW', 'P_0':0.022*u.keV/u.cm**3, 'r_p':466.8*u.kpc, 'a':1.8, 'b':3.1, 'c':0.0}
clust.set_density_crp_isodens_scal_param(scal=1.0)
clust.abundance = 0.3
clust.set_magfield_isodens_scal_param(4.7*u.uG, scal=0.5)
clust.spectrum_crp_model = {'name':'ExponentialCutoffPowerLaw', 'Index':1.8, 'CutoffEnergy':30*u.TeV}
clust.spectrum_cre1_model = {'name': 'PowerLaw', 'Index':3.0}
clust.X_crp_E = {'X':0.01, 'R_norm':clust.R500}
clust.X_cre1_E = {'X':0.00001, 'R_norm':clust.R500}

clust.Epmax = 10e6*u.TeV
clust.Eemax = 1e-2*u.TeV

### Check what are the model parameters

In [ ]:
# clust.print_param() # To print the parameters in the terminal
clust.save_param()  # To write the parameters in the output directory

In [ ]:
def plot_CTA_sens(): #CTA north 50h sensitivity
    bins_lo = [0.02, 0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126] # E_min (TeV)
    bins_hi = [0.0316, 0.0501, 0.0794, 0.126, 0.2, 0.316, 0.501, 0.794, 1.26, 2, 3.16, 5.01, 7.94, 12.6, 20, 31.6, 50.1, 79.4, 126, 200] # E_max (TeV)
    data = [2.37e-11, 8.35e-12, 2.02e-12, 9.41e-13, 5.37e-13, 3.58e-13, 2.43e-13, 1.76e-13, 1.35e-13, 1.24e-13, 1.1e-13,
            1.21e-13, 1.44e-13, 2.06e-13, 3.2e-13, 5.1e-13, 7.97e-13, 1.2e-12, 1.93e-12, 3.92e-12] # S (erg / cm^2 / s)
    data_GeV = [i * 624.15064799632 for i in data]
    mid_point = [np.exp((np.log(i) + np.log(j)) / 2) * 1000 for i, j in zip(bins_lo, bins_hi)]
    plt.loglog(mid_point, data_GeV, color='k', ls='-.', lw=float(2.0), alpha=1, label = 'CTA North sensivity 50h')

def plot_LHAASO_sens(): #LHAASO sensitivity
    bins_lo = [0.3, 0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300] # E_min (TeV)
    bins_hi = [0.6, 1, 2, 2.5, 3, 5, 8, 10, 20, 50, 100, 170, 300, 600] # E_max (TeV)
    data = [3e-9, 8e-10, 4e-10, 1e-10, 8e-11, 7e-11, 8e-11, 9e-11, 6e-11, 4e-11, 1.5e-11, 2e-11, 5e-11, 7e-11 ] # S (GeV/ cm^2 / s)
    mid_point = [np.exp((np.log(i) + np.log(j)) / 2) * 1000 for i, j in zip(bins_lo, bins_hi)]
    plt.loglog(mid_point, data, color='gray', ls='-.', lw=float(2.0), alpha=1, label = 'LHAASO sensitivity')

def plot_fermi_sens(): # Fermi sensitivity
    bins_lo = [5e-5, 7e-5, 2e-4, 4e-4, 6e-4, 1.5e-3, 2e-3, 4e-3, 8e-3,5e-2, 0.1] #in TeV
    bins_hi = [7e-5, 2e-4, 4e-4, 6e-4, 1.5e-3, 2e-3, 4e-3, 8e-3, 5e-2,0.1, 0.3] #in TeV
    data = [6e-12, 3e-12, 1e-12, 8e-13, 6.5e-13, 3.75e-13, 3e-13, 3.5e-13, 5e-13, 2e-12, 4.25e-12] # S (erg / cm^2 / s)
    data_GeV = [i * 624.15064799632 for i in data]
    mid_point = [np.exp((np.log(i) + np.log(j)) / 2) * 1000 for i, j in zip(bins_lo, bins_hi)]
    plt.loglog(mid_point, data_GeV, color='grey', ls='-.', lw=float(2.0), alpha=0.5, label = 'Fermi sensitivity')
    
def plot_fermi_coma():
    energy = np.logspace(-1, 1, 100)*u.GeV
    F23_fermi = 5.2e-9
    gamma23 = 2.3
    F20_fermi = 3.2e-9 #ph cm-2 s-1
    gamma20 = 2.0
    N20_fermi = F20_fermi / scipy.integrate.quad(lambda e: e ** (-gamma20), 0.1, 10)[0]
    N23_fermi = F23_fermi / scipy.integrate.quad(lambda e: e ** (-gamma23), 0.1, 10)[0]
    dNdE_20 = N20_fermi * energy.value ** (-gamma20) * u.Unit('GeV-1 cm-2 s-1')
    dNdE_23 = N23_fermi * energy.value ** (-gamma23) * u.Unit('GeV-1 cm-2 s-1')
    plt.fill_between(energy, dNdE_20 * energy ** 2, dNdE_23 * energy ** 2, facecolor ='k', alpha=0.3, 
                     label='Fermi upper limit')

# Plot gamma spectrum

In [ ]:
energy = np.logspace(-1,5,100)*u.GeV


plt.figure(0,figsize=(10,8))
plt.ylim(1e-12,1e-8)
plt.xlim(1e-1,1e6)

clust.set_density_crp_isodens_scal_param(scal=1.0)
scal = clust._density_crp_model['beta'] / clust._density_gas_model['beta']

def plot_gamma_spec(a, b, c, X=0.01):
    clust.X_crp_E['X'] = X
    clust.X_cre1_E['X'] = X * 1e-3
    for i,j in zip([a, b, c],['-','--','-.']):
        clust.spectrum_crp_model['Index'] = i
        e1, s1 = clust.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
        plt.loglog(e1, (e1**2*s1).to('GeV cm-2 s-1'), ls=j, c = 'red', lw = float(3.0), label='$\gamma$ = ' + 
                   str(clust.spectrum_crp_model['Index']))
        
def plot_gamma_spec_tot(a, b, c, d, e, f, X=0.01):
    clust.X_crp_E['X'] = X
    clust.X_cre1_E['X'] = X * 1e-3
    for i,j in zip([(a, d), (b, e), (c, f)],['-','--','-.']):
        clust.spectrum_crp_model['Index'] = i[0]
        e1, s1 = clust.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
        clust.spectrum_crp_model['Index'] = i[1]
        e2, s2 = clust.get_gamma_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
        plt.loglog(e1, (e1**2*(s1+s2)).to('GeV cm-2 s-1'), ls=j, c = 'blue', lw = float(3.0), label='$\gamma_1$ = ' + 
                   str(i[0]) + ', $\gamma_2$ =' + str(i[1]))
        
plot_fermi_coma()
plot_CTA_sens()
plot_LHAASO_sens()
# plot_fermi_sens()
# plot_gamma_spec(2.0, 2.3, 2.7, X=0.02) #будує спектр одної компоненти (для троьх гамма)
plot_gamma_spec_tot(1.4, 1.6, 1.8, 2.0, 2.3, 2.7, X=0.01) #будує сумарний спектр двох компонент
plt.text(0.01 * plt.gca().get_xlim()[1], 0.5 * plt.gca().get_ylim()[1], '$X_{cr,p} =$ ' + str(clust.X_crp_E['X']), 
         fontsize = 18)
plt.text(0.01 * plt.gca().get_xlim()[1], 0.3 * plt.gca().get_ylim()[1], '$\eta_{CR,p} =$ ' + str(scal), 
         fontsize = 18)
plt.xlabel('Energy (GeV)')
plt.ylabel('Flux ($GeV/cm^2/s$)')
plt.legend()
try:
    # збереження рисунку в файл. в папку clust.output_dir
    # plt.savefig(clust.output_dir + '/gamma_spec_Xcrp_' + str(clust.X_crp_E['X']) + '_eta_'+ str(scal) +'.png')
    plt.savefig(clust.output_dir + '/gamma_spec_tot_Xcrp_' + str(clust.X_crp_E['X']) + '_eta_'+ str(scal) +'.png')
    print('Figure saved')
except:
    print('Error while saving figure')

# Plot neutrino spectrum

In [ ]:
energy = np.logspace(-1,6,100)*u.GeV

plt.figure(1,figsize=(10,8))
plt.ylim(1e-12,1e-8)
plt.xlim(1e-1,1e6)

def plot_neutrino_spec(a, b, c, X=0.01):
    clust.X_crp_E['X'] = X
    clust.X_cre1_E['X'] = X * 1e-3
    for i,j in zip([a, b, c],['-','--','-.']):
        clust.spectrum_crp_model['Index'] = i
        e1, s1 = clust.get_neutrino_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
        plt.loglog(e1, (e1**2*s1).to('GeV cm-2 s-1'), ls=j, c = 'red', lw = float(3.0), label='$\gamma$ = ' + 
                   str(clust.spectrum_crp_model['Index']))

def plot_neutrino_spec_tot(a, b, c, d, e, f, X=0.01):
    clust.X_crp_E['X'] = X
    clust.X_cre1_E['X'] = X * 1e-3
    for i,j in zip([(a, d), (b, e), (c, f)],['-','--','-.']):
        clust.spectrum_crp_model['Index'] = i[0]
        e1, s1 = clust.get_neutrino_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
        clust.spectrum_crp_model['Index'] = i[1]
        e2, s2 = clust.get_neutrino_spectrum(energy,Rmax=clust.R_truncation, type_integral='spherical')
        plt.loglog(e1, (e1**2*(s1+s2)).to('GeV cm-2 s-1'), ls=j, c = 'blue', lw = float(3.0), label='$\gamma_1$ = ' + 
                   str(i[0]) + ', $\gamma_2$ =' + str(i[1]))    
    
# plot_neutrino_spec(1.4, 1.6, 1.8, X=0.01) #будує спектр одної компоненти (для троьх гамма)
plot_neutrino_spec_tot(1.4, 1.6, 1.8, 2.0, 2.3, 2.7, X=0.01) #будує сумарний спектр двох компонент
plt.plot(np.logspace(3,6,10), [1e-9] * 10, c = 'k', lw = float(3.0), ls='--', label = 'IceCube discovery potential')
plt.text(0.01 * plt.gca().get_xlim()[1], 0.5 * plt.gca().get_ylim()[1], '$X_{cr,p} =$ ' + str(clust.X_crp_E['X']), 
         fontsize = 18)
plt.text(0.01 * plt.gca().get_xlim()[1], 0.3 * plt.gca().get_ylim()[1], '$\eta_{CR,p} =$ ' + str(scal), 
         fontsize = 18)
plt.xlabel('Energy (GeV)')
plt.ylabel('Flux ($GeV/cm^2/s$)')
plt.legend()
try:
    # збереження рисунку в файл. в папку clust.output_dir
    # plt.savefig(clust.output_dir + '/neutrino_spec_Xcrp_' + str(clust.X_crp_E['X']) + '_eta_'+ str(scal) +'.png')
    plt.savefig(clust.output_dir + '/neutrino_spec_tot_Xcrp_' + str(clust.X_crp_E['X']) + '_eta_'+ str(scal) +'.png')
    print('Figure saved')
except:
    print('Error while saving figure')

In [ ]:
import sys
import astropy
import scipy
import pickle
import healpy
import matplotlib
print('python = ' + sys.version)
print('astropy = ' + astropy.__version__)
print('numpy = ' + np.__version__)
print('scipy = ' + scipy.__version__)
print('matplotlib = ' + matplotlib.__version__)
print('pickle = ' + pickle.format_version)
print('healpy = ' + healpy.__version__)